ドキュメント
https://qiita.com/api/v2/docs

ページング(page)の最大数は100,1ページ当たりの件数(per_page)最大数も100

In [ ]:
# FIXME:取得中に記事が追加されるとエラーになる問題
# 記事取得→DB格納→一意制約違反となる

In [ ]:
import os
import requests
import asyncpg
import asyncio
import nest_asyncio
from dateutil import parser
from datetime import timezone

DATABASE_URL = "postgresql://{}:{}@postgres-service:{}/{}".format(os.environ['POSTGRES_USER'], os.environ['POSTGRES_PASSWORD'], os.environ['POSTGRES_SERVICE_SERVICE_PORT'], os.environ['POSTGRES_DB'])

def fetch_qiita_articles(token, page=1, per_page=100):
    """Qiitaの記事を取得する"""
    url = f'https://qiita.com/api/v2/items?page={page}&per_page={per_page}'
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return None

async def insert_articles(articles):
    conn = await asyncpg.connect(DATABASE_URL)
    try:
        async with conn.transaction():
            await conn.executemany("INSERT INTO learning_resources (title, url, created_date) VALUES ($1, $2, $3)",
                                   [(article['title'], article['url'], parser.parse(article['created_at']).replace(tzinfo=None)) for article in articles])
    except Exception as e:
        print("データベースエラー:", e)
    finally:
        await conn.close()

async def insert_articles_rigid(articles):
    # いっそunique消すか？→消して取得後にURL重複してるやつの片方削除としたほうがio的にはいいのでは？
    # TODO:記事を取得してDBに格納している間に1件でも追加されると一意制約エラーとなる
    # 一意制約違反の場合にそのレコードのみ除外されるようにexecutemanyでなくexecuteを使用
    # ioが100倍になる。よっぽど厳格にやりたいならこちら。今回は別にいいかな。
    """記事のリストをデータベースに挿入する"""
    conn = await asyncpg.connect(DATABASE_URL)
    for article in articles:
        try:
            async with conn.transaction():
                await conn.execute(
                    "INSERT INTO learning_resources (title, url, created_date) VALUES ($1, $2, $3)",
                    article['title'], article['url'], parser.parse(article['created_at']).replace(tzinfo=None)
                )
        except Exception as e:
            print(f"データベースエラー: {e} - {article['url']}")
    await conn.close()

async def main():
    token = os.environ['QIITA_TOKEN']
    for i in range(100):
        articles = fetch_qiita_articles(token, page=i)
        if articles:
            await insert_articles(articles)

if __name__ == "__main__":
    # asyncio.run(main())
    # notebookでの実行時はnestを許可する
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

データベースエラー: duplicate key value violates unique constraint "learning_resources_url_key"
DETAIL:  Key (url)=(https://qiita.com/ksgiksg/items/191a3e38a1103831d132) already exists.
